# Direct Frequency Topic Identification (DFTI)

One of the challenges with using LDA is that it is an unsupervised algorithm.  As a result, like PCA, the clusters that it discovers may or may not align with desired goals of data scientists.  As an example consider the following text for review:

"My dog really likes this cat food made from fish products.  Most dog food he has tried hasn't gone over well with him at all.  But this product is really outstanding."

In the above scenario, this text might be classified as either dog, cat or fish.  It could also be classified as food or product depending on how the LDA system is configured.

As a way to get around this problem, a different approach could be used.  If, instead of starting with the review text, what if we started with buckets.  Such as the following:

dog
cat
fish
bird
rabbit
turtle
snake
rat
mouse
hamster
gerbil
reptile
horse
other

Now, we might be able to create our own guess at "categorizing" the reviews by counting the number of times these words appear in a document, and then using that information to drive the probability of category.  For example

| ASIN | dog | cat | fish | etc | 
|---|---|---|---|---|
| 123456778 | 2 | 1 | 1 | ... |

The following code will attempt to do this for the products in the data set.  


In [1]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict


CONN = sqlite3.connect('pets_all.db')
buckets = ['dog','cat','fish','bird','amphibians and reptiles','rabbits and rodents','farm animals']

words = {
'dog' : ['dog', 'dogs', 'puppy', 'pup', 'puppies', 'paw', 'paws'],
'cat' : ['cat', 'cats', 'kitten', 'kittens', 'feline', 'felines'],
'fish' : ['goldfish', 'fish', 'fishes', 'koi', 'angelfish', 'catfish', 'guppies', 'freshwater', 'saltwater'],
'bird' : ['parrot','parrots','beak', 'feather','beaks', 'feathers', 'bird', 'birds', 'canary', 'canaries','budgerigar','budgerigars','budgies', 'parakeets','cockatiel','cockatiels','cockatoo','cockatoos','macaw','macaws', 'dove','doves'],
'rabbits and rodents' : ['rabbit', 'rabbits', 'bunny', 'bunnies', 'hare', 'hares','lop','rat','mouse','hamster','gerbil','rats','mice','hamsters','gerbils', 'rodents', 'rodent', 'guinea', 'ferret', 'hedgehog','ferrets', 'hedgehogs'],
# 'turtle' : ['turtle', 'turtles', 'tortoise', 'tortoises', 'cooter', 'loggerhead','snapper', 'terrapin','leatherback'],
# 'rodent' : ['rat','mouse','hamster','gerbil','rats','mice','hamsters','gerbils', 'rodents', 'rodent', 'guinea', 'ferret', 'hedgehog','ferrets', 'hedgehogs'],
# 'rabbit' : ['rabbit', 'rabbits', 'bunny', 'bunnies', 'hare', 'hares','lop'],
'farm animals': ['chicken','chicks','chickens', 'duck', 'ducks','goat', 'goats', 'bull','cow','bulls','cows','lamb','sheep', 'sheeps','horse','pony','horses','ponies', 'stud','mare','studs','mares'],
'amphibians and reptiles':['turtle', 'turtles', 'tortoise', 'tortoises','tadpole', 'tadpoles', 'cooter', 'loggerhead','snapper', 'terrapin','leatherback', 'snake', 'snakes', 'gecko', 'geckos','lizard', 'lizards', 'frog', 'frogs', 'toad', 'toads', 'amphibian', 'amphibians', 'reptile', 'reptiles','snake','snakes', 'dragon', 'dragons']
}

# print(words)

cur = CONN.cursor()

cur.execute("SELECT asin from reviews")
asins = cur.fetchall()
print("Number of reviews: {}".format(len(asins)))

cur.execute("SELECT DISTINCT asin from reviews")
counts=[]
unique_asins = cur.fetchall()
print("Unique asins found: {}".format(len(unique_asins)))
start=datetime.now()

i=0


def get_title():
    cur.execute("SELECT asin, title, description from products")
    titles = cur.fetchall()
    results_df = pd.DataFrame(titles)
    results_df.columns = ['asin', 'title','description']
    results_df["combined"] = results_df["title"] + results_df["description"]
    result_dict = dict(zip(results_df.asin, results_df.combined))
    return result_dict

product_description = get_title()

for asin in unique_asins:
    i=i+1
    prog = i/len(unique_asins)*100
    
    if prog % 10 == 0:
        print("{}%".format(prog))
    
    sql = "select review_text from reviews where asin = '{}'".format(asin[0])
    cur.execute(sql)
    rec = [asin[0]]
    full_text = " ".join([x[0] for x in cur.fetchall()])
    if (asin[0] in product_description):
        full_text = full_text + product_description[asin[0]]
    for bucket in buckets:
        cnt = sum(full_text.lower().count(x) for x in words[bucket])
        rec.append(cnt)
    counts.append(rec)
       
print("Compute time {}".format(datetime.now()-start))
            
results_df = pd.DataFrame(counts)
# display(results_df.head(20))



def get_max_row(x):
    vals = x.values[1:]
    if max(vals)==0:
        return -1
    else:
        return(np.argmax(vals))

def get_topic(row, b):
    x = row['topic_id']
    if x==-1:
        return "other"
    else:
        return b[x]

results_df.to_csv("temp_results.csv")
    
results_df['topic_id'] = results_df.apply(lambda row: get_max_row(row), axis=1)
results_df['topic'] = results_df.apply(lambda row: get_topic(row, buckets), axis=1)

results_df.rename(columns={0:'asin'}, inplace=True)
results_df = results_df[['asin','topic_id','topic']]
display(results_df.head(10))
results_df.to_csv('category.csv', index=False)
results_df.groupby('topic').count()

print("Done.")

Number of reviews: 6542483
Unique asins found: 198402
50.0%
100.0%
Compute time 0:05:12.716043


,asin,topic_id,topic
0,061539972X,0,dog
1,0615553605,0,dog
2,0760339597,0,dog
3,0793816793,2,fish
4,0972585419,3,bird
5,0975412868,0,dog
6,0977264815,0,dog
7,0978619404,5,rabbits and rodents
8,0979699967,6,farm animals
9,0982083513,-1,other


Done.


# Let's see how we did.  

In this cell we will take a random sample of 10 rows from the dataset and see how our grouping compares to that of the product description



In [ ]:
# sample_df = results_df.sample(n=20)

def get_product(asin):
    cur.execute("Select title from products where asin = '{}'".format(asin))
    product = cur.fetchall()
    return product[0]

# for idx, row in sample_df.iterrows():
#     print(get_product(row['asin']))
#     print(row['topic'])
#     print("\n\n")
    
def get_title():
    cur.execute("SELECT asin, title, description from products LIMIT 10")
#     cur.execute("SELECT * from sqlite_master WHERE tbl_name = 'products' AND type = 'table'")
    titles = cur.fetchall()
    results_df = pd.DataFrame(titles)
    results_df.columns = ['asin', 'title','description']
    results_df["combined"] = results_df["title"] + results_df["description"]
    result_dict = dict(zip(results_df.asin, results_df.combined))
    return result_dict
    

result_dict = get_title()
counts=[]
i=0

for asin in unique_asins:
    i=i+1
    prog = i/len(unique_asins)*100
    rec = [asin[0]]
    if (asin[0] in result_dict):
        full_text = result_dict[asin[0]]
        for bucket in buckets:
            cnt = sum(full_text.lower().count(x) for x in words[bucket])
            rec.append(cnt)
    else:
        for bucket in buckets:
            cnt = 0
            rec.append(cnt)
        
    counts.append(rec)
    
results_df = pd.DataFrame(counts)
display(results_df.head(20))
